# Tratamiento de datos masivos

## Práctica 3

- Se valorará la claridad del código y evitar redundancias o código poco eficiente; en particular se valorará utilizar las funciones para RDDs de Spark minimizando el uso de Python. 
- Además de las funciones que se piden se pueden añadir otras auxiliares si se necesitan, y también otros imports
- El código debe funcionar correctamente no solo con las pruebas que vienen de ejemplo sino con cualquier otra prueba



#### Setup

In [4]:
import os.path
from subprocess import check_call
import importlib
import os
import sys

modules = ["findspark", "pyspark"]


if "google.colab" in sys.modules:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    spark = "spark-3.2.0-bin-hadoop3.2.tgz"
    if not os.path.isfile(spark):
        !wget -q https://downloads.apache.org/spark/spark-3.2.0/{spark}
        !tar xf {spark}
        os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
        os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

print("Checking required modules")
for m in modules:
    torch_loader = importlib.util.find_spec(m)
    if torch_loader is not None:
        print(m, " found")
    else:
        print(m, " not found, installing")
        if "google.colab" in sys.modules:
            check_call(["pip", "install", "-q", m])
        else:
            check_call([sys.executable, "-m", "pip", "install", "--user", m])
print("Done!")
#!pip install -q findspark
#  !pip install -q pyspark
print("Ready!")

Checking required modules
findspark  found
pyspark  found
Done!
Ready!


Solo por si alguien necesita ajusta variables de entorno desde Python; en ese caso descomentar este código y sustituirlo por código propio

In [5]:
"""
%matplotlib inline
import os
# cambiamos las variables del sistema
spark = 'C:/hlocal/tdm/spark/spark-3.2.0-bin-hadoop3.2'
if not(os.path.isdir(spark+"/bin")) or not(os.path.isdir(spark+"/jars")) :
        print("Error, la carpeta en 'spark' debe contener los directorios bin y jars ")
else:
        # en el path se añade
        #path = os.environ.get('PATH')
        #path = path+ ';'+spark+'\\bin;'
        #os.environ['PATH'] = path
        os.environ['SPARK_HOME']= spark
        os.environ['HADOOP_HOME']= spark
        os.environ['PYSPARK_DRIVER_PYTHON']= 'jupyter'
        os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='notebook'

        # si da problema con collect quizás haya que poner java_home a la localización de java 8
        #os.environ['JAVA_HOME']= 'C:\\Program Files\\Java\\jdk1.8.0_151'
        #os.environ['PATH'] = os.environ.get('JAVA_HOME')+'\\bin;'+spark
        print("Hecho")
print("Preparado!!")

"""

'\n%matplotlib inline\nimport os\n# cambiamos las variables del sistema\nspark = \'C:/hlocal/tdm/spark/spark-3.2.0-bin-hadoop3.2\'\nif not(os.path.isdir(spark+"/bin")) or not(os.path.isdir(spark+"/jars")) :\n        print("Error, la carpeta en \'spark\' debe contener los directorios bin y jars ")\nelse:\n        # en el path se añade\n        #path = os.environ.get(\'PATH\')\n        #path = path+ \';\'+spark+\'\\bin;\'\n        #os.environ[\'PATH\'] = path\n        os.environ[\'SPARK_HOME\']= spark\n        os.environ[\'HADOOP_HOME\']= spark\n        os.environ[\'PYSPARK_DRIVER_PYTHON\']= \'jupyter\'\n        os.environ[\'PYSPARK_DRIVER_PYTHON_OPTS\']=\'notebook\'\n\n        # si da problema con collect quizás haya que poner java_home a la localización de java 8\n        #os.environ[\'JAVA_HOME\']= \'C:\\Program Files\\Java\\jdk1.8.0_151\'\n        #os.environ[\'PATH\'] = os.environ.get(\'JAVA_HOME\')+\'\\bin;\'+spark\n        print("Hecho")\nprint("Preparado!!")\n\n'

In [6]:
import findspark

findspark.init()

import pyspark  # only run after findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext


df = spark.sql("""select 'spark' as hola """)
df.show()

24/10/08 20:05:53 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 10.9.111.218 instead (on interface wlp0s20f3)
24/10/08 20:05:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/08 20:05:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+
| hola|
+-----+
|spark|
+-----+



## Carga de datos
El siguiente código descarga de internet un fichero con datos de contaminación de la ciudad de madrid. 

La estructura esperada del fichero es 

PROVINCIA;MUNICIPIO;ESTACION;MAGNITUD;PUNTO_MUESTREO;ANO;MES;DIA;H01;V01;H02;V02;H03;V03;H04;V04;H05;V05;H06;V06;H07;V07;H08;V08;H09;V09;H10;V10;H11;V11;H12;V12;H13;V13;H14;V14;H15;V15;H16;V16;H17;V17;H18;V18;H19;V19;H20;V20;H21;V21;H22;V22;H23;V23;H24;V24

Las parejas de las forma Hi;Vi indican la medición en la hora i (valor Hi) y si esta medición es válida (si lo es Vi será 'V' y si no 'N'). Nótese que el fichero no tiene cabecera.

In [7]:
file = "./pollutionfix.csv"  # change the local folder if needed

raw_data = sc.textFile(file).cache()  # RDD of strings
print("Rows: ", raw_data.count())
raw_data.take(5)

Rows:  2875


['28;079;4;1;28079004_1_38;2018;04;01;00002;V;00002;V;00002;V;00001;V;00002;V;00002;V;00003;V;00003;V;00003;V;00004;V;00003;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00003;V;00004;V;00003;V',
 '28;079;4;1;28079004_1_38;2018;04;02;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00003;V;00007;V;00006;V;00005;V;00003;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V',
 '28;079;4;1;28079004_1_38;2018;04;03;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V',
 '28;079;4;1;28079004_1_38;2018;04;04;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V;00002;V',
 '28;079;4;1;28079004_1_38;2018;04;05;00002;V;00002;V;00002;V;00002;V;00

**Ejercicio 1** Sospechamos que hay filas vacías ('') en el conjunto de datos. 

a) Escribir código para comprobar cuántas filas son iguales a ''

b) Generar un nuevo RDD `raw2` en el que se han eliminado estas filas vacías (quedarás 2872 elementos en raw2)

In [8]:
# a) Sol.
raw_data.filter(lambda x: x == "").count()

4

In [9]:
# b) Sol
raw2 = raw_data.filter(lambda x: x != "")

#
raw2.count()

2871

**Ejercicio 2** raw2 es un RDD de strings; queremos convertirlo en un RDD de listas de strings, es decir separar sus componentes (recordar que la función split divide un string según un carácter de separación). Llamar al resultado raw3


In [10]:
raw3 = raw2.map(lambda x: x.split(";"))
raw3.take(5)

[['28',
  '079',
  '4',
  '1',
  '28079004_1_38',
  '2018',
  '04',
  '01',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00001',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00003',
  'V',
  '00003',
  'V',
  '00003',
  'V',
  '00004',
  'V',
  '00003',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00003',
  'V',
  '00004',
  'V',
  '00003',
  'V'],
 ['28',
  '079',
  '4',
  '1',
  '28079004_1_38',
  '2018',
  '04',
  '02',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00003',
  'V',
  '00007',
  'V',
  '00006',
  'V',
  '00005',
  'V',
  '00003',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
  '00002',
  'V',
 

24/10/08 20:06:05 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


**Ejercicio 3** Las posiciones de las columnas V01, V02...V24 deberían tener o bien el valor V el valor N. Escribir una función con las siguientes características:

- name: countUnexpected
- desc: count el número de filas de la columna que se le pasa como parámetro en la que no hay ni 'V' no 'N'
- param:
    - rdd: un RDD de listas de strings
    - pos: una posición, es decir un número de columna
- return: el número de filas del rdd que en la posición pos no tiene ni 'N' ni 'V'


In [11]:
def countUnexpected(rdd, pos):
    return rdd.filter(lambda x: x[pos] not in ("V", "N")).count()

Para probar

In [12]:
columns = "PROVINCIA;MUNICIPIO;ESTACION;MAGNITUD;PUNTO_MUESTREO;ANO;MES;DIA;H01;V01;H02;V02;H03;V03;H04;V04;H05;V05;H06;V06;H07;V07;H08;V08;H09;V09;H10;V10;H11;V11;H12;V12;H13;V13;H14;V14;H15;V15;H16;V16;H17;V17;H18;V18;H19;V19;H20;V20;H21;V21;H22;V22;H23;V23;H24;V24".split(
    ";"
)
posV01 = columns.index("V01")

countUnexpected(raw3, posV01)  # 0

3

**Ejercicio 4**

Ahora queremos contar cuántos valores distintos de N,V hay entre todas las columnas V01....V24. Para ello:
    
- Primero definimos una lista `posV` con todas las posiciones de estas columnas (eso ya se da hecho) 
- Ahora utilizaremos un bucle para ir aplicando la función del ejercicio anterior a cada columna cuya posición está en `poss` y a raw3, e iremos sumando el resultado en una variable `total`

Valor esperado: 7

In [13]:
posV = [posV01 + 2 * i for i in range(24)]
# sol
total = 0

for i in posV:
    total += countUnexpected(raw3, i)

print(total)

7


**Ejercicio 5** Ahora queremos eliminar todos las filas que tengan algún valor no válido (distinto de 'V') en cualquiera de las columnas de `poss`. Para ello repetimos la idea anterior:

a) Primero escribir una función removeNonValid(rdd,pos)
- name: removeNonValid
- desc: Elimina las filas con algún dato no válido
- param:
    - rdd: un RDD de listas de strings
    - pos: una posición, es decir un número de columna
- return: el rdd de entrada pero quedándose solo con los elementos que en la posición pos  tengan una 'V'



In [14]:
def removeNonValid(rdd, pos):
    return rdd.filter(lambda x: x[pos] == "V")

In [15]:
# para probar
raw3 = removeNonValid(raw3, posV01)
raw3.count()  # 2487

2847

b) Ahora hacer un bucle que llame repetidamente a removeNonValid para cada posición en poss. Recordar asignar el resultado a raw3 en cada ocasión para que el resultado sea acumulativo 

In [16]:
for i in posV:
    raw3 = removeNonValid(raw3, i)

raw3.count()  # 2629

2629

**Ejercicio 6** Repetir la misma idea pero para suprimir todas las filas en las que el valor Hi no sea un número real. 

- Escribir primero código para crear la lista de posiciones de los valores H, llamar a la variable `posH`

- Escribir una función `removeNonNumber(rdd,pos)` que devuelve un RDD solo con las filas que tengan en la posición p un número (se puede/debe usar como auxiliar la función is_number(x) que se incluye a continuación).

- Hacer un bucle llamando a removeNonNumber con raw3 y cada posición de `posH`.

In [17]:
def is_number(n):
    try:
        float(n)  # Type-casting the string to `float`.
    # If string is not a valid `float`,
    # it'll raise `ValueError` exception
    except ValueError:
        return False
    return True


posH = [i - 1 for i in posV]


#
def removeNonNumber(rdd, pos):
    return rdd.filter(lambda x: is_number(x[pos]))


# bucle aplicando la función
for i in posH:
    raw3 = removeNonNumber(raw3, i)

#
raw3.count()  # 2627

2627

**Ejercicio 7** Dado que ahora ya todas las filas de `posH`so números reales válidos, generar código para convertir 
los strings de las posiciones en posH en float (con la función predefinida float(x)). Llamar raw4 al RDD resultante.
Se valorará hacerlo reduciendo al mínimo el número de recorridos de raw3 (algo que hasta ahora no hemos tenido en cuenta)

In [18]:
raw4 = raw3.map(lambda x: [float(y) if i in posH else y for i, y in enumerate(x)])

raw4.take(5)

[['28',
  '079',
  '4',
  '1',
  '28079004_1_38',
  '2018',
  '04',
  '01',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  1.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  3.0,
  'V',
  3.0,
  'V',
  3.0,
  'V',
  4.0,
  'V',
  3.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  3.0,
  'V',
  4.0,
  'V',
  3.0,
  'V'],
 ['28',
  '079',
  '4',
  '1',
  '28079004_1_38',
  '2018',
  '04',
  '02',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  3.0,
  'V',
  7.0,
  'V',
  6.0,
  'V',
  5.0,
  'V',
  3.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V'],
 ['28',
  '079',
  '4',
  '1',
  '28079004_1_38',
  '2018',
  '04',
  '03',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',
  2.0,
  'V',


**Ejercicio 8** Las columnas posV ya no nos valen para nada, ya que en raw4 todos los datos son válidos. Escribir código que genere un nuevo RDD raw5 a partir de raw4 que ya no contenga ninguna de las columnas en posV. De nuevo se valorará hacerlo en el menor número posible de recorridos

In [19]:
raw5 = raw4.map(lambda x: [y for i, y in enumerate(x) if i not in posV])

raw5.take(5)

[['28',
  '079',
  '4',
  '1',
  '28079004_1_38',
  '2018',
  '04',
  '01',
  2.0,
  2.0,
  2.0,
  1.0,
  2.0,
  2.0,
  3.0,
  3.0,
  3.0,
  4.0,
  3.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  3.0,
  4.0,
  3.0],
 ['28',
  '079',
  '4',
  '1',
  '28079004_1_38',
  '2018',
  '04',
  '02',
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  3.0,
  7.0,
  6.0,
  5.0,
  3.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0],
 ['28',
  '079',
  '4',
  '1',
  '28079004_1_38',
  '2018',
  '04',
  '03',
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0],
 ['28',
  '079',
  '4',
  '1',
  '28079004_1_38',
  '2018',
  '04',
  '04',
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0],
 ['28',
  '079',
  '

**Ejercicio 9** Queremos cambiar el RDD `raw5`  para que cada hora sea un elemento nuevo del RDD. Por ejemplo el elemento

    ['28',  '079',  '4',  '1',  '28079004_1_38',  '2018',  '04',  '01',  2.0,  50.0,  20.0,...3.0]
    
se convertira en 24 elementos, uno por hora (nótese que se incluye una nueva columna con el número de hora)    

    ['28',  '079',  '4',  '1',  '28079004_1_38',  '2018',  '04',  '01', 1, 2.0]
    ['28',  '079',  '4',  '1',  '28079004_1_38',  '2018',  '04',  '01', 2, 50.0]
    ['28',  '079',  '4',  '1',  '28079004_1_38',  '2018',  '04',  '01', 3, 20.0]
    ...
    ['28',  '079',  '4',  '1',  '28079004_1_38',  '2018',  '04',  '01', 24, 3.0]


Llamar `raw6` al nuevo rdd

In [20]:
raw6 = raw5.zipWithIndex().map(lambda x: [*x[0][:8], x[1] + 1, x[0][8]])

raw6.take(30)

[['28', '079', '4', '1', '28079004_1_38', '2018', '04', '01', 1, 2.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '02', 2, 2.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '03', 3, 2.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '04', 4, 2.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '05', 5, 2.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '07', 6, 2.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '08', 7, 2.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '11', 8, 1.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '13', 9, 1.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '14', 10, 3.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '15', 11, 2.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '16', 12, 2.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '17', 13, 2.0],
 ['28', '079', '4', '1', '28079004_1_38', '2018', '04', '18', 14, 5.0],
 

**Ejercicio 10** Calcular la media de contaminación (el dato de la última columna) por punto de muestreo (columna 5ª, pos 4) y hora (columna 9ª, pos 8). Se deben usar recursos de Spark y el resultado debe salir por orden de punto de muestreo  y dentro del mismo punto de muestreo por hora. La salida será de la forma:


    [('28079004_12_8', 1, 51.857142857142854),
     ('28079004_12_8', 2, 41.285714285714285),
     ('28079004_12_8', 3, 35.285714285714285),
     ('28079004_12_8', 4, 30.75),
     ('28079004_12_8', 5, 26.607142857142858),
     ('28079004_12_8', 6, 29.035714285714285),
     ('28079004_12_8', 7, 44.642857142857146),
     ('28079004_12_8', 8, 91.25),
     ('28079004_12_8', 9, 129.46428571428572),
     ('28079004_12_8', 10, 131.42857142857142),
     ('28079004_12_8', 11, 86.39285714285714),
     ...
     ('28079004_1_38', 1, 2.269230769230769),
     ('28079004_1_38', 2, 2.1153846153846154),
     ('28079004_1_38', 3, 2.0384615384615383),
     ('28079004_1_38', 4, 2.0),
     ('28079004_1_38', 5, 1.9230769230769231),
     ...

In [55]:
import numpy as np

raw7 = raw5.groupBy(lambda x: x[4]).map(
    lambda x: (
        x[0],
        np.mean(np.transpose(list(map(lambda y: y[8:], x[1]))), axis=1),
    )
)

raw7.take(100)

[('28079004_1_38',
  array([2.26923077, 2.11538462, 2.03846154, 2.        , 1.92307692,
         1.96153846, 2.19230769, 2.88461538, 3.57692308, 3.46153846,
         2.53846154, 2.19230769, 1.92307692, 1.69230769, 1.61538462,
         1.69230769, 1.73076923, 1.80769231, 1.76923077, 1.76923077,
         1.96153846, 2.11538462, 2.26923077, 2.42307692])),
 ('28079004_7_8',
  array([12.82758621,  9.10344828,  7.03448276,  6.5862069 ,  5.79310345,
          6.06896552, 12.44827586, 35.72413793, 52.4137931 , 49.89655172,
         24.44827586, 11.34482759,  6.86206897,  4.55172414,  3.10344828,
          2.03448276,  1.68965517,  1.79310345,  1.79310345,  3.20689655,
          5.10344828,  9.27586207, 11.17241379, 14.13793103])),
 ('28079004_8_8',
  array([34.92857143, 29.82142857, 26.92857143, 21.89285714, 18.53571429,
         20.17857143, 24.89285714, 38.07142857, 50.32142857, 54.64285714,
         50.28571429, 44.96428571, 41.75      , 37.64285714, 33.07142857,
         28.46428571, 26.82